In [ ]:
import gspread
import pandas as pd
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from helpers.connection import get_google_credentials
from gspread.exceptions import WorksheetNotFound
from pytz import timezone
import locale


# ==============================
# 1. Helper Functions
# ==============================
def get_first_advent(year):
    """Find the first Advent Sunday (last Sunday before Christmas)."""
    dec_25 = datetime(year, 12, 25)
    # Go backward to Sunday
    days_to_sunday = dec_25.weekday() + 1  # Monday=0..Sunday=6
    first_advent = dec_25 - timedelta(days=days_to_sunday + 21)  # 4 Sundays before Christmas
    return first_advent


def liturgical_year(date):
    """Determine the Liturgical Year (A/B/C) based on a given date."""
    year = date.year
    first_advent = get_first_advent(year)

    if date >= first_advent:
        lit_year = year + 1  # move to next liturgical year
    else:
        lit_year = year

    # Mapping cycle: 2020 = A → lit_year % 3
    # 2020 % 3 = 1 → A
    # 2021 % 3 = 2 → B
    # 2022 % 3 = 0 → C
    mapping = {1: "A", 2: "B", 0: "C"}
    return mapping[lit_year % 3]


def save_df_to_gsheet(spreadsheet, worksheet_output_name, df):
    """
    Save DataFrame to a Google Sheets worksheet.

    Params:
        spreadsheet (gspread.Spreadsheet): target spreadsheet object
        worksheet_output_name (str): worksheet/tab name
        df (pandas.DataFrame): data to be written
    """
    # 1. Get worksheet or create a new one if not exists
    try:
        sheet_out = spreadsheet.worksheet(worksheet_output_name)
    except WorksheetNotFound:
        sheet_out = spreadsheet.add_worksheet(
            title=worksheet_output_name,
            rows=str(len(df) + 10),
            cols=str(len(df.columns) + 5)
        )

    # 2. Clear old content
    sheet_out.clear()

    # 3. Prepare data (header + DataFrame content)
    data = [df.columns.tolist()] + df.astype(str).values.tolist()

    # 4. Additional metadata
    tz = timezone("Asia/Jakarta")
    last_update_str = f"Last Update: {datetime.now(tz).strftime('%d-%b-%Y %H:%M:%S WIB')}"

    today = datetime.today()
    bulan, tahun = today.month, today.year
    url = f"https://www.imankatolik.or.id/kalender.php?b={bulan}&t={tahun}"

    # 5. Combine all updates into batch_update (single API call)
    requests = [
        {
            "range": f"A1:{chr(65+len(df.columns)-1)}{len(df)+1}",
            "values": data
        },
        {"range": "K1", "values": [[last_update_str]]},
        {"range": "K2", "values": [["Liturgical Calendar:"]]},
        {"range": "L2", "values": [[url]]}
    ]

    sheet_out.batch_update(requests)

    print(f"✅ Data successfully saved to sheet: {worksheet_output_name}")


# ==============================
# 2. Google Sheets Connection
# ==============================
SPREADSHEET_ID = "1xMNjbpQJhh8jTOaNlxPWy9B2nTEMBAURR9Ys3O90jlM"  # Source Sheet ID
WORKSHEET_NAME = "Jadwal Pasdior"  # Source Worksheet
SPREADSHEET_ID_OUTPUT = "1nqY5jNzJvsy7v37jnb-rlSDUNvsLYiuHq5-ryAW1Kxs"  # Target Sheet ID
WORKSHEET_OUTPUT = "jadwal"  # Target Worksheet (default)
ORGANIST_WORKSHEET_NAME = "Data Organis"

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]
creds = get_google_credentials(scope)
client = gspread.authorize(creds)


# ==============================
# 3. Load Organist List
# ==============================
organist_sheet = client.open_by_key(SPREADSHEET_ID_OUTPUT).worksheet(ORGANIST_WORKSHEET_NAME)
all_organist_data = organist_sheet.get_all_values()  # all rows and columns with values
names = [row[0] for row in all_organist_data[1:]]  # skip header
clean_organist_list_name = [name.lower() for name in names]


# ==============================
# 4. Load and Preprocess Data
# ==============================
sheet = client.open_by_key(SPREADSHEET_ID).worksheet(WORKSHEET_NAME)
all_data = sheet.get_all_values()  # all rows and columns with values

# Slice: start from row 5 (index 4) and columns B–K (index 1 to 10)
data = [row[1:11] for row in all_data[4:] if len(row) >= 11]
df = pd.DataFrame(data, columns=["B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]).copy()

# Update F & G columns based on J & K
mask_j = df["J"].astype(str).str.strip() != ""
df.loc[mask_j, "F"] = df.loc[mask_j, "J"]
df.loc[mask_j, ["F", "G"]] = df.loc[mask_j, ["J", "K"]].values


# ==============================
# 5. Extra Data (O–R)
# ==============================
data_extra = [row[14:18] for row in all_data[4:] if len(row) >= 18]
df_extra = pd.DataFrame(data_extra, columns=["O", "P", "Q", "R"]).copy()

# Map to main format
df_extra["B"] = df_extra["O"]
df_extra["C"] = df_extra["P"]
df_extra["F"] = df_extra["Q"]
df_extra["G"] = df_extra["R"]
df_extra["D"] = ""
df_extra["E"] = ""
df_extra = df_extra[["B", "C", "D", "E", "F", "G"]].copy()

# Merge
df_all = pd.concat([df[["B", "C", "D", "E", "F", "G"]], df_extra], ignore_index=True)


# ==============================
# 6. Filter & Clean Dates
# ==============================
month_map = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
    "Sep": "09", "Sept": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}

# Clean and parse dates
b_str = df_all["B"].astype(str).str.strip()
b_str_num = b_str.replace(month_map, regex=True)
b_dt = pd.to_datetime(b_str_num, dayfirst=True, errors="coerce")

# Fallback: Excel serial numbers
serial_mask = b_str.str.match(r"^\d{4,6}$", na=False)
b_dt.loc[serial_mask] = (
    pd.to_datetime("1899-12-30") +
    pd.to_timedelta(b_str.loc[serial_mask].astype(int), unit="D")
)

df_all.loc[:, "B_dt"] = b_dt

# Filter for today or later
today_jkt = datetime.now(ZoneInfo("Asia/Jakarta")).date()
df_all = df_all[df_all["B_dt"].dt.date >= today_jkt].copy()

# Sort by date
df_all = df_all.sort_values(by="B_dt").reset_index(drop=True)


# ==============================
# 7. Final Output Format
# ==============================
df_clean = df_all[["B", "C", "D", "E", "F", "G", "B_dt"]].copy()
df_clean.columns = ["Tanggal", "Jam", "Anamnesis", "Cara Tobat", "Koor", "Organis", "tgl-format"]

# Add liturgical year (A/B/C)
df_clean["Tahun Liturgi"] = df_clean["tgl-format"].apply(lambda x: liturgical_year(x))

# Day names in Indonesian
try:
    locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
except:
    try:
        locale.setlocale(locale.LC_TIME, "id_ID")
    except:
        pass

df_clean["Hari"] = df_clean["tgl-format"].dt.strftime("%A")

# Reorder columns
df_clean = df_clean[["Hari", "Tanggal", "Jam", "Anamnesis", "Cara Tobat", "Koor", "Organis", "Tahun Liturgi"]]

# Add Weekday flag (yes/no)
df_clean["Weekday"] = df_clean["Hari"].apply(
    lambda x: "yes" if x not in ["Sabtu", "Minggu", "Saturday", "Sunday"] else "no"
)


# ==============================
# 8. Save Results to Google Sheets
# ==============================
spreadsheet = client.open_by_key(SPREADSHEET_ID_OUTPUT)

for name in clean_organist_list_name:
    filter_df = df_clean[df_clean["Organis"].str.lower() == name].copy()
    save_df_to_gsheet(spreadsheet=spreadsheet, worksheet_output_name="Jadwal " + name.capitalize(), df=filter_df)

# ==============================
# 9. Final Log
# ==============================
print("🎉 All data successfully updated and saved to Google Sheets.")


✅ Data successfully saved to sheet: Jadwal Vicky
✅ Data successfully saved to sheet: Jadwal Jun
✅ Data successfully saved to sheet: Jadwal Ratri
✅ Data successfully saved to sheet: Jadwal Maureen
✅ Data successfully saved to sheet: Jadwal Prescyl
✅ Data successfully saved to sheet: Jadwal Doni
✅ Data successfully saved to sheet: Jadwal Irena
✅ Data successfully saved to sheet: Jadwal Vincent
✅ Data successfully saved to sheet: Jadwal Maxien
🎉 All data successfully updated and saved to Google Sheets.


In [4]:
import sys
from babel.dates import format_date
import gspread
import pandas as pd
import asyncio
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
from pytz import timezone
from gspread.exceptions import WorksheetNotFound
from dotenv import load_dotenv, find_dotenv
import locale
import nest_asyncio

# =======================================
# SETUP LINGKUNGAN & IMPORT BANTUAN
# =======================================
sys.path.append("..")
from utils.telegram_bot import TelegramBot
from helpers.connection import get_google_credentials

load_dotenv(find_dotenv())
nest_asyncio.apply()

# =======================================
# 1. HELPER FUNCTIONS
# =======================================
def get_first_advent(year):
    dec_25 = datetime(year, 12, 25)
    days_to_sunday = dec_25.weekday() + 1
    return dec_25 - timedelta(days=days_to_sunday + 21)

def liturgical_year(date):
    year = date.year
    first_advent = get_first_advent(year)
    lit_year = year + 1 if date >= first_advent else year
    mapping = {1: "A", 2: "B", 0: "C"}
    return mapping[lit_year % 3]

def save_df_to_gsheet(spreadsheet, worksheet_output_name, df):
    try:
        sheet_out = spreadsheet.worksheet(worksheet_output_name)
    except WorksheetNotFound:
        sheet_out = spreadsheet.add_worksheet(
            title=worksheet_output_name,
            rows=str(len(df) + 10),
            cols=str(len(df.columns) + 5)
        )

    sheet_out.clear()
    data = [df.columns.tolist()] + df.astype(str).values.tolist()

    tz = timezone("Asia/Jakarta")
    last_update_str = f"Last Update: {datetime.now(tz).strftime('%d-%b-%Y %H:%M:%S WIB')}"
    today = datetime.today()
    url = f"https://www.imankatolik.or.id/kalender.php?b={today.month}&t={today.year}"

    requests = [
        {"range": f"A1:{chr(65+len(df.columns)-1)}{len(df)+1}", "values": data},
        {"range": "K1", "values": [[last_update_str]]},
        {"range": "K2", "values": [["Liturgical Calendar:"]]},
        {"range": "L2", "values": [[url]]}
    ]
    sheet_out.batch_update(requests)
    print(f"✅ Disimpan ke Google Sheet: {worksheet_output_name}", flush=True)


# =======================================
# 2. GOOGLE SHEETS CONNECTION
# =======================================
SPREADSHEET_ID = "1xMNjbpQJhh8jTOaNlxPWy9B2nTEMBAURR9Ys3O90jlM"
WORKSHEET_NAME = "Jadwal Pasdior"
SPREADSHEET_ID_OUTPUT = "1nqY5jNzJvsy7v37jnb-rlSDUNvsLYiuHq5-ryAW1Kxs"
WORKSHEET_OUTPUT = "jadwal"
ORGANIST_WORKSHEET_NAME = "Data Organis"

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive"
]
creds = get_google_credentials(scope)
client = gspread.authorize(creds)

# =======================================
# 3. LOAD ORGANIST LIST
# =======================================
organist_sheet = client.open_by_key(SPREADSHEET_ID_OUTPUT).worksheet(ORGANIST_WORKSHEET_NAME)
all_organist_data = organist_sheet.get_all_values()

organist_records = []
for row in all_organist_data[1:]:
    if not row or not row[0].strip():
        continue
    name = row[0].strip()
    chat_id = row[1].strip() if len(row) > 1 and row[1].strip() else None
    organist_records.append({"name": name, "chat_id": chat_id})

clean_organist_list_name = [r["name"].lower() for r in organist_records]


# =======================================
# 4. LOAD & PREPROCESS DATA
# =======================================
sheet = client.open_by_key(SPREADSHEET_ID).worksheet(WORKSHEET_NAME)
all_data = sheet.get_all_values()

data = [row[1:11] for row in all_data[4:] if len(row) >= 11]
df = pd.DataFrame(data, columns=["B", "C", "D", "E", "F", "G", "H", "I", "J", "K"]).copy()

mask_j = df["J"].astype(str).str.strip() != ""
df.loc[mask_j, ["F", "G"]] = df.loc[mask_j, ["J", "K"]].values

data_extra = [row[14:18] for row in all_data[4:] if len(row) >= 18]
df_extra = pd.DataFrame(data_extra, columns=["O", "P", "Q", "R"])
df_extra["B"], df_extra["C"], df_extra["F"], df_extra["G"] = df_extra["O"], df_extra["P"], df_extra["Q"], df_extra["R"]
df_extra["D"], df_extra["E"] = "", ""
df_extra = df_extra[["B", "C", "D", "E", "F", "G"]]
df_all = pd.concat([df[["B", "C", "D", "E", "F", "G"]], df_extra], ignore_index=True)

month_map = {
    "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
    "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
    "Sep": "09", "Sept": "09", "Oct": "10", "Nov": "11", "Dec": "12"
}
b_str = df_all["B"].astype(str).str.strip().replace(month_map, regex=True)
b_dt = pd.to_datetime(b_str, dayfirst=True, errors="coerce")

serial_mask = b_str.str.match(r"^\d{4,6}$", na=False)
b_dt.loc[serial_mask] = pd.to_datetime("1899-12-30") + pd.to_timedelta(b_str.loc[serial_mask].astype(int), unit="D")

df_all["B_dt"] = b_dt
today_jkt = datetime.now(ZoneInfo("Asia/Jakarta")).date()
df_all = df_all[df_all["B_dt"].dt.date >= today_jkt].copy().sort_values("B_dt").reset_index(drop=True)

df_clean = df_all[["B", "C", "D", "E", "F", "G", "B_dt"]].copy()
df_clean.columns = ["Tanggal", "Jam", "Anamnesis", "Cara Tobat", "Koor", "Organis", "tgl-format"]
df_clean["Tahun Liturgi"] = df_clean["tgl-format"].apply(liturgical_year)

try:
    locale.setlocale(locale.LC_TIME, "id_ID.UTF-8")
except:
    try:
        locale.setlocale(locale.LC_TIME, "id_ID")
    except:
        pass

# df_clean["Hari"] = df_clean["tgl-format"].dt.strftime("%A")
df_clean["Hari"] = df_clean["tgl-format"].apply(
    lambda d: format_date(d, "EEEE", locale="id") if pd.notnull(d) else ""
)
df_clean["Weekday"] = df_clean["Hari"].apply(lambda x: "yes" if x not in ["Sabtu", "Minggu", "Saturday", "Sunday"] else "no")

# =======================================
# 5. SEND TELEGRAM REMINDERS
# =======================================
spreadsheet = client.open_by_key(SPREADSHEET_ID_OUTPUT)


async def send_telegram_reminders():
    print("🚀 Memulai proses pengiriman reminder...\n", flush=True)

    for rec in organist_records:
        name, chat_id = rec["name"], rec["chat_id"]
        print(f"🔹 Memproses {name}...", flush=True)

        # Filter jadwal untuk organis ini
        filter_df = df_clean[df_clean["Organis"].str.lower() == name.lower()].copy()
        await asyncio.to_thread(save_df_to_gsheet, spreadsheet, f"Jadwal {name.capitalize()}", filter_df)

        if not filter_df.empty:
            # Ambil 3 jadwal terdekat
            next_three = filter_df.head(3).copy()
            next_three["Tanggal_dt"] = next_three["tgl-format"]

            # Format tanggal dan jam (pakai Babel untuk Bahasa Indonesia)
            tanggal_list = []
            for _, row in next_three.iterrows():
                if pd.notnull(row["Tanggal_dt"]):
                    hari = format_date(row["Tanggal_dt"], "EEEE", locale="id")
                    tanggal = format_date(row["Tanggal_dt"], "d MMMM y", locale="id")
                    jam = str(row["Jam"]).strip() if pd.notnull(row["Jam"]) else ""
                    tanggal_list.append(f"- {hari}, {tanggal} • {jam}")

            reminder_text = (
                f"Hi {name.capitalize()}, jadwal organis berikutnya adalah:\n" +
                "\n".join(tanggal_list)
            )

            print(reminder_text, flush=True)
            print("=" * 60, flush=True)

            # Kirim ke Telegram jika ada chat_id
            if chat_id:
                try:
                    bot = TelegramBot(chat_id=chat_id)
                    await bot.send(reminder_text)
                    print(f"📨 Reminder dikirim ke {name} ({chat_id})", flush=True)
                except Exception as e:
                    print(f"⚠️ Gagal kirim ke {name}: {e}", flush=True)

        await asyncio.sleep(2)  # jeda antar user

    print("\n✅ Semua reminder selesai dikirim!", flush=True)

# =======================================
# 6. JALANKAN
# =======================================
await send_telegram_reminders()


🚀 Memulai proses pengiriman reminder...

🔹 Memproses Vicky...
✅ Disimpan ke Google Sheet: Jadwal Vicky
Hi Vicky, jadwal organis berikutnya adalah:
- Minggu, 16 November 2025 • 10.00
- Minggu, 23 November 2025 • 17.00
- Minggu, 14 Desember 2025 • 13:00
🔹 Memproses Jun...
✅ Disimpan ke Google Sheet: Jadwal Jun
Hi Jun, jadwal organis berikutnya adalah:
- Minggu, 14 Desember 2025 • 17.00
- Minggu, 21 Desember 2025 • 17.00
- Minggu, 4 Januari 2026 • 17.00
🔹 Memproses Ratri...
✅ Disimpan ke Google Sheet: Jadwal Ratri
Hi Ratri, jadwal organis berikutnya adalah:
- Minggu, 7 Desember 2025 • 10.00
- Sabtu, 3 Januari 2026 • 17.00
- Minggu, 1 Februari 2026 • 10.00
🔹 Memproses Maureen...
✅ Disimpan ke Google Sheet: Jadwal Maureen
Hi Maureen, jadwal organis berikutnya adalah:
- Sabtu, 15 November 2025 • 17.00
- Minggu, 30 November 2025 • 12:00
- Minggu, 30 November 2025 • 10.00
🔹 Memproses Prescyl...
✅ Disimpan ke Google Sheet: Jadwal Prescyl
Hi Prescyl, jadwal organis berikutnya adalah:
- Minggu, 3